# BART XLSX analysis pipeline (BIDS-compatible)
This notebook reads the BART `.xlsx` files produced by your PsychoPy task (sheets: `trials`, `pumps`, `summary`).
It outputs cleaned per-trial data, per-session metrics, and (optionally) longitudinal trends across sessions.


In [ ]:
!pip -q install openpyxl scipy

import re
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from scipy import stats


In [ ]:
import os, glob, time
OUTPUT_DIR = '/content/bart_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)
RUN_TAG = time.strftime('%Y%m%d-%H%M%S')

# Descriptive subject tag for output filenames (computed later after data load)
def compute_sub_tag(df, col="sub"):
    import pandas as pd
    if df is None or col not in df.columns:
        return "sub-UNKNOWN"
    subs = [str(s).upper().strip().replace("SUB-","") for s in df[col].dropna().unique()]
    subs = sorted([s for s in subs if s])
    if len(subs) == 0:
        return "sub-UNKNOWN"
    if len(subs) == 1:
        return f"sub-{subs[0]}"
    return f"sub-{subs[0]}-{subs[-1]}_n{len(subs)}"
print('OUTPUT_DIR:', OUTPUT_DIR, 'RUN_TAG:', RUN_TAG)


OUTPUT_DIR: /content/bart_data RUN_TAG: 20260120-202310


## 1) Upload files
Upload **one or more** BART `.xlsx` files.

**Optional:** also upload a `manifest.csv` with columns: `sub`, `condition` (NF/SHAM).

In [ ]:
# Use XLSX files already present in /content (Colab file pane)
import glob, os

# only BART files (BIDS-like)
xlsx_files = sorted([p for p in glob.glob("/content/*.xlsx")
                     if "sub-" in os.path.basename(p).lower()])

manifest_csv = sorted(glob.glob("/content/manifest*.csv"))
manifest_xlsx = sorted(glob.glob("/content/manifest*.xlsx"))

manifest_path = manifest_csv[0] if manifest_csv else (manifest_xlsx[0] if manifest_xlsx else None)
print("Manifest:", manifest_path)


print("BART XLSX files:", len(xlsx_files))
print("\n".join(xlsx_files[:10]))

print('First few XLSX:', xlsx_files[:10])

assert len(xlsx_files) > 0, 'No .xlsx files found in /content. Drag them into the Colab file pane first.'


Manifest: /content/manifest.xlsx
BART XLSX files: 8
/content/sub-P004_ses-S001_task-BART_run-001_beh.xlsx
/content/sub-P004_ses-S002_task-BART_run-001_beh.xlsx
/content/sub-P004_ses-S003_task-BART_run-001_beh.xlsx
/content/sub-P004_ses-S004_task-BART_run-001_beh.xlsx
/content/sub-P005_ses-S001_task-BART_run-001_beh.xlsx
/content/sub-P005_ses-S002_task-BART_run-001_beh.xlsx
/content/sub-P005_ses-S003_task-BART_run-001_beh.xlsx
/content/sub-P005_ses-S004_task-BART_run-001_beh.xlsx
First few XLSX: ['/content/sub-P004_ses-S001_task-BART_run-001_beh.xlsx', '/content/sub-P004_ses-S002_task-BART_run-001_beh.xlsx', '/content/sub-P004_ses-S003_task-BART_run-001_beh.xlsx', '/content/sub-P004_ses-S004_task-BART_run-001_beh.xlsx', '/content/sub-P005_ses-S001_task-BART_run-001_beh.xlsx', '/content/sub-P005_ses-S002_task-BART_run-001_beh.xlsx', '/content/sub-P005_ses-S003_task-BART_run-001_beh.xlsx', '/content/sub-P005_ses-S004_task-BART_run-001_beh.xlsx']


## 2) Helpers: parse BIDS filename + load sheets

In [ ]:
BIDS_PAT = re.compile(
    r"sub-(?P<sub>[^_]+)_ses-(?P<ses>[^_]+)_task-(?P<task>[^_]+)_run-(?P<run>[^_]+)_beh\.xlsx$",
    re.IGNORECASE
)

def parse_bids(fn: str):
    m = BIDS_PAT.search(fn.replace(" ", ""))
    if not m:
        # fallback: try without _beh
        m2 = re.search(r"sub-([^_]+)_ses-([^_]+)_task-([^_]+)_run-([^_]+)", fn, re.IGNORECASE)
        if not m2:
            raise ValueError(f"Filename not BIDS-like: {fn}")
        return {"sub": m2.group(1), "ses": m2.group(2), "task": m2.group(3), "run": m2.group(4)}
    return {k: v.upper() for k, v in m.groupdict().items()}

def read_bart_xlsx(fn: str):
    meta = parse_bids(fn)
    xl = pd.ExcelFile(fn)
    trials = pd.read_excel(xl, sheet_name="trials") if "trials" in xl.sheet_names else pd.DataFrame()
    pumps  = pd.read_excel(xl, sheet_name="pumps")  if "pumps"  in xl.sheet_names else None
    summary= pd.read_excel(xl, sheet_name="summary") if "summary" in xl.sheet_names else None

    # Attach filename meta (in case your sheet doesn't include it)
    for df in [trials, pumps, summary]:
        if df is None or len(df)==0:
            continue
        for k,v in meta.items():
            if k not in df.columns:
                df[k]=v
        df["file"]=fn
    return meta, trials, pumps, summary

def to_numeric_safe(df, cols):
    for c in cols:
        if c in df.columns:
            df[c]=pd.to_numeric(df[c], errors="coerce")
    return df


## 3) Load all XLSX into one table

In [ ]:
all_trials=[]
all_pumps=[]
all_summary=[]

for fn in xlsx_files:
    meta, trials, pumps, summary = read_bart_xlsx(fn)
    if len(trials):
        all_trials.append(trials)
    if pumps is not None and len(pumps):
        all_pumps.append(pumps)
    if summary is not None and len(summary):
        all_summary.append(summary)

trials_df = pd.concat(all_trials, ignore_index=True) if all_trials else pd.DataFrame()
pumps_df  = pd.concat(all_pumps,  ignore_index=True) if all_pumps  else None
summary_df= pd.concat(all_summary,ignore_index=True) if all_summary else None

print("Trials rows:", len(trials_df))
print("Pumps rows :", 0 if pumps_df is None else len(pumps_df))
print("Summary rows:", 0 if summary_df is None else len(summary_df))

trials_df.head()


# Compute descriptive subject tag once data are loaded
SUB_TAG = compute_sub_tag(trials_df, col='sub')
print('SUB_TAG:', SUB_TAG)


Trials rows: 360
Pumps rows : 5022
Summary rows: 16
SUB_TAG: sub-P004-P005_n2


## 4) Optional: merge condition (NF vs SHAM) from manifest
If you uploaded a `manifest.csv` with columns `sub, condition`, it will be joined by `sub`.
If you did not upload one, condition is left blank.

In [ ]:
if manifest_files:
    mf = pd.read_csv(manifest_files[0])
    mf.columns = [c.strip().lower() for c in mf.columns]
    # accept either 'sub' or 'subject' naming
    if "sub" not in mf.columns and "subject" in mf.columns:
        mf = mf.rename(columns={"subject":"sub"})
    if "condition" in mf.columns:
        mf["condition"]=mf["condition"].astype(str).str.upper().str.strip()
    mf["sub"]=mf["sub"].astype(str).str.upper().str.strip().str.replace("^SUB-","", regex=True)

    trials_df["sub"]=trials_df["sub"].astype(str).str.upper().str.strip().str.replace("^SUB-","", regex=True)
    trials_df = trials_df.merge(mf[["sub","condition"]], on="sub", how="left")
    if pumps_df is not None:
        pumps_df["sub"]=pumps_df["sub"].astype(str).str.upper().str.strip().str.replace("^SUB-","", regex=True)
        pumps_df = pumps_df.merge(mf[["sub","condition"]], on="sub", how="left")
else:
    trials_df["condition"] = np.nan
    if pumps_df is not None:
        pumps_df["condition"] = np.nan

trials_df[["sub","ses","run","task","condition","file"]].drop_duplicates().head(10)


NameError: name 'manifest_files' is not defined

## 5) Clean trials + compute session metrics
We compute (Main block):
- mean adjusted pumps (non-exploded trials only)
- explosion frequency
- final bank (max total_earnings)
- pump latency (median/mean)
- resting theta pre/post if present


In [ ]:
df = trials_df.copy()

# numeric conversions (safe if missing)
df = to_numeric_safe(df, [
    "exploded_int","pump_count","adjusted_pumps_trial",
    "pump_latency_first","pump_latency_mean","pump_latency_median",
    "trial_duration_sec","trial_earnings","total_earnings",
    "rest_pre_eo_theta_mean","rest_pre_ec_theta_mean",
    "rest_post_eo_theta_mean","rest_post_ec_theta_mean",
    "baseline_mu","baseline_sigma","baseline_n"
])

# standardize block filtering
df["block"] = df["block"].astype(str)

# Valid-trial filter (tweak as needed)
if "trial_duration_sec" in df.columns:
    df["valid_trial"] = df["trial_duration_sec"].between(2, 180, inclusive="both")
else:
    df["valid_trial"] = True

# Latency validity window (decision hesitancy)
LAT_MIN, LAT_MAX = 0.15, 6.0
if "pump_latency_median" in df.columns:
    df["latency_ok"] = df["pump_latency_median"].between(LAT_MIN, LAT_MAX, inclusive="both")
else:
    df["latency_ok"] = True

# focus on Main block
main = df[df["block"].str.lower().eq("main") & df["valid_trial"]].copy()

# adjusted pumps: only non-exploded trials
if {"exploded_int","adjusted_pumps_trial"}.issubset(main.columns):
    main["adjusted_pumps_clean"] = np.where(
        (main["exploded_int"]==0) & main["latency_ok"],
        main["adjusted_pumps_trial"],
        np.nan
    )

# Session-level metrics per (sub,ses,run)
group_cols = ["sub","ses","run","task","condition","file"]
def session_metrics(g):
    out={}
    out["n_trials_main"] = len(g)
    out["explosion_frequency"] = g["exploded_int"].mean() if "exploded_int" in g.columns else np.nan
    out["mean_adjusted_pumps"] = g["adjusted_pumps_clean"].mean() if "adjusted_pumps_clean" in g.columns else np.nan
    out["median_pump_latency"] = g["pump_latency_median"].median() if "pump_latency_median" in g.columns else np.nan
    out["mean_pump_latency"] = g["pump_latency_mean"].mean() if "pump_latency_mean" in g.columns else np.nan
    out["final_bank"] = g["total_earnings"].max() if "total_earnings" in g.columns else np.nan

    # Pull rest/baseline values (they're repeated across rows; take first non-null)
    for col in ["baseline_mu","baseline_sigma","baseline_n",
                "rest_pre_eo_theta_mean","rest_pre_ec_theta_mean",
                "rest_post_eo_theta_mean","rest_post_ec_theta_mean"]:
        if col in g.columns:
            s = g[col].dropna()
            out[col] = s.iloc[0] if len(s) else np.nan
        else:
            out[col] = np.nan
    return pd.Series(out)

sessions = main.groupby(group_cols, dropna=False).apply(session_metrics).reset_index()
sessions = sessions.sort_values(["sub","ses","run"])
sessions.head(10)


## 6) Longitudinal trends across sessions (optional)
If each subject has multiple sessions, compute a simple **slope per subject** over session number.
This works even if you keep condition blinded (or absent).

In [ ]:
# Try to convert ses like 'S032' -> 32 for ordering
def ses_to_int(s):
    s = str(s)
    m = re.search(r"(\d+)", s)
    return int(m.group(1)) if m else np.nan

sessions["ses_num"] = sessions["ses"].apply(ses_to_int)

def slope_over_sessions(g, metric):
    d = g.sort_values("ses_num")
    y = d[metric].astype(float).values
    x = d["ses_num"].astype(float).values
    mask = ~np.isnan(x) & ~np.isnan(y)
    if mask.sum() < 3:
        return np.nan
    X = np.vstack([x[mask], np.ones(mask.sum())]).T
    b, a = np.linalg.lstsq(X, y[mask], rcond=None)[0]  # y = b*x + a
    return b

slopes=[]
for (sub, cond), g in sessions.groupby(["sub","condition"], dropna=False):
    slopes.append({
        "sub": sub,
        "condition": cond,
        "n_sessions": g["ses_num"].nunique(),
        "slope_latency": slope_over_sessions(g, "median_pump_latency"),
        "slope_adjusted": slope_over_sessions(g, "mean_adjusted_pumps"),
        "slope_explosion": slope_over_sessions(g, "explosion_frequency"),
        "slope_bank": slope_over_sessions(g, "final_bank"),
    })
slopes_df = pd.DataFrame(slopes)
slopes_df.head(10)


## 7) Quick plots

In [ ]:
def plot_metric(metric):
    plt.figure()
    for label, g in sessions.groupby('condition', dropna=False):
        means = g.groupby('ses_num')[metric].mean()
        plt.plot(means.index.values, means.values, marker='o', label=str(label))
    plt.xlabel('Session')
    plt.ylabel(metric)
    plt.title(f'{metric} across sessions')
    plt.legend()
    # Save figure into bart_data instead of downloading
    out_png = os.path.join(OUTPUT_DIR, f'bart_{metric}_across_sessions_{SUB_TAG}_{RUN_TAG}.png')
    plt.savefig(out_png, dpi=150, bbox_inches='tight')
    plt.show()
    plt.close()

for m in ['mean_adjusted_pumps','explosion_frequency','median_pump_latency','final_bank']:
    if m in sessions.columns:
        plot_metric(m)


## 8) Export combined tables

In [ ]:
# Write cleaned tables into /content/bart_data with descriptive filenames
trials_out   = os.path.join(OUTPUT_DIR, f'bart_trials_main_clean_{SUB_TAG}_{SUB_TAG}_{RUN_TAG}.csv')
sessions_out = os.path.join(OUTPUT_DIR, f'bart_sessions_metrics_{SUB_TAG}_{SUB_TAG}_{RUN_TAG}.csv')
slopes_out   = os.path.join(OUTPUT_DIR, f'bart_subject_slopes_{SUB_TAG}_{SUB_TAG}_{RUN_TAG}.csv')

main.to_csv(trials_out, index=False)
sessions.to_csv(sessions_out, index=False)
slopes_df.to_csv(slopes_out, index=False)

print('Wrote:')
print(' -', trials_out)
print(' -', sessions_out)
print(' -', slopes_out)
